In [3]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
from textblob import TextBlob
import re

%store -r

In [4]:
# Import the restored data
%run PrepareData.ipynb

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


(3717964, 7)

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL


(4336445, 2)

,tweet_id,ticker_symbol
3239577,550442977802207232,TSLA
3239578,550443807834402816,TSLA
3239579,550443808606126081,TSLA
3239580,550443809700851716,TSLA
3239581,550443857142611968,TSLA


(1096868, 2)

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num
0,550442977802207232,TSLA,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
1,550443807834402816,TSLA,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
2,550443808606126081,TSLA,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
3,550443809700851716,TSLA,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
4,550443857142611968,TSLA,Cprediction,1420071016,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num
1096863,1212158772015034369,TSLA,StockJoke,1577836146,"I'm getting wasted on NYE, and thanks to @NHTS...",0,0,0
1096864,1212159099632267268,TSLA,sweetog76,1577836224,$TSLA Very Interesting dude:,0,0,0
1096865,1212159184931717120,TSLA,MelaynaLokosky,1577836245,If only 2020 brought more self-awareness to Te...,0,1,0
1096866,1212159838882533376,TSLA,ShortingIsFun,1577836401,In 2020 I may start Tweeting out positive news...,0,0,1
1096867,1212160015332728833,TSLA,Commuternyc,1577836443,Patiently Waiting for the no twitter sitter tw...,0,0,5


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096868 entries, 0 to 1096867
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   tweet_id       1096868 non-null  int64 
 1   ticker_symbol  1096868 non-null  object
 2   writer         1073202 non-null  object
 3   post_date      1096868 non-null  int64 
 4   body           1096868 non-null  object
 5   comment_num    1096868 non-null  int64 
 6   retweet_num    1096868 non-null  int64 
 7   like_num       1096868 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 75.3+ MB
Stored 'tsla_tweets_df' (DataFrame)
Stored 'tsla_stock_values_df' (DataFrame)


In [5]:
# Reset the index of the dataframe
tsla_tweets_df = tsla_tweets_df.reset_index()

In [7]:
# Clean the text
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)#Remove @mentions
    text = re.sub(r'#', '', text)# Remove '#' hash tag
    text = re.sub(r'RT[\s]+', '', text)# Remove RT
    text = re.sub(r'http?:\/\/\S+', '', text)# Remove hyperlink
    text = re.sub(r'\$[A-Za-z0-9]+', '', text)# Remove $
    text = re.sub(r'https?:\/\/\S+', '', text)# Remove hyperlink
    
    return text

tsla_tweets_df['body'] = tsla_tweets_df['body'].apply(cleanTxt)

In [9]:
# Creaet a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [10]:
# Apply the models to the cleaned data
tsla_tweets_df['Subjectivity'] = tsla_tweets_df['body'].apply(getSubjectivity)
tsla_tweets_df['Polarity'] = tsla_tweets_df['body'].apply(getPolarity)

In [11]:
# Check the sentiment analysis result
tsla_tweets_df[['Subjectivity','Polarity']]

,Subjectivity,Polarity
0,0.400000,-1.500000e-01
1,0.100000,0.000000e+00
2,0.477273,3.806818e-01
3,0.000000,0.000000e+00
4,0.300000,1.000000e+00
...,...,...
357537,0.666667,1.809524e-01
357538,0.733333,1.850372e-17
357539,0.850000,4.000000e-01
357540,0.333333,1.666667e-01


In [12]:
# Get the mean
tsla_sentiments_df_textblob = tsla_tweets_df[["Subjectivity","Polarity"]].copy()
tsla_sentiments_df_textblob['Date'] = pd.to_datetime(tsla_tweets_df['post_date'].dt.date, utc=False)
tsla_sentiments_df_textblob = tsla_sentiments_df_textblob.groupby('Date').mean()
display(tsla_sentiments_df_textblob.head())
display(tsla_sentiments_df_textblob.tail())

,Subjectivity,Polarity
Date,,
2015-01-01,0.400000,-0.150000
2015-01-02,0.406539,0.180581
2015-01-03,0.210845,0.046556
2015-01-04,0.201275,0.076732
2015-01-05,0.078016,0.031528


,Subjectivity,Polarity
Date,,
2019-12-28,0.386012,0.079870
2019-12-29,0.382562,0.072515
2019-12-30,0.392603,0.080258
2019-12-31,0.404880,0.081927
2020-01-01,0.415301,0.123758


In [ ]:
%store tsla_sentiments_df_textblob